# 강화학습에 이용하기위한 데이터세트 생성하기 (Q-Table)

# 0. 아무것도 안한 오리지널 코드

In [ ]:
import random
import math

total = int(input("얼마를 충전하시겠습니까? "))

jackpot_num_list = []


previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:

        num = random.randint(1, 125)
        if num == 1:
                    return jackpot_num
        else:
            jackpot_num += 1


def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget # global로 해주어야 밖으로 반환됨

    jackpot_num = make_jackpot_num()

    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100 # 포아송과 기하분포의 cdf로 계산된 확률
        
        # 20% 기준으로 구간 설정하여 분포확인하기
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)  # 구간별 카운트를 0으로 초기화
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:  # 숫자가 해당 구간에 속하는지 확인
                    interval_counts[i] += 1
                    break  # 해당 구간에 숫자가 속하면 다음 숫자로 넘어감
        # jackpot_num_list의 길이가 0보다 큰 경우에만 평균을 계산합니다.
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0  # 또는 적절한 기본값
        
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)        
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else: # 이렇게도 되는구나 신기하다.
            start = -7  # 시작 범위
            end = 7     # 끝 범위
            gap = 0.5   # 간격
            value = 2   # 초기 대응값
            # -7부터 7까지 0.5 간격으로 범위 설정
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break  # 조건에 맞는 값을 찾으면 반복문 탈출
                start += gap  # 다음 범위로 업데이트
                value += 1  # 다음 값으로 업데이트
        
        #사라지는구간 찾기
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None  # 초기값 설정(없으면 오류)
        if len(jackpot_num_list) >= 6:  # 6번째 요소가 존재하는지 확인
            sixth_element = jackpot_num_list[5]  # 6번째 요소를 가져옴
            for index, (start, end) in enumerate(ranges, start=1):  # enumerate를 사용하여 각 구간의 인덱스를 얻습니다.
                if start <= sixth_element <= end:
                    forget = index

        
        print("\n")
        #print(bul_r)
        print(f"사라지는구간{forget}")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        #print(bul)
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        #print(f"이전 잭팟까지의 횟수 = {jackpot_num_list[-5:]}")
        
        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()

# 1. txt로 저장하는 코드

In [ ]:
import random
import math

# 파일에 데이터를 저장하는 함수를 수정합니다.
def save_betting_to_file(interval_counts, probability, bul, forget, betting): # 여기에 써있는 변수 함수에 적용됨 betting만 써있을때 오류였음
    with open("betting_history.txt", "a") as file:  # 'a'는 파일이 존재할 경우 내용을 끝에 추가
        file.write(f"{interval_counts}{probability:.2f}{bul}{forget}{betting}\n")  # 다양한 데이터를 파일에 쓰고, 줄바꿈

total = int(input("얼마를 충전하시겠습니까? "))

jackpot_num_list = []


previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:

        num = random.randint(1, 125)
        if num == 1:
                    return jackpot_num
        else:
            jackpot_num += 1


def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget # global로 해주어야 밖으로 반환됨

    jackpot_num = make_jackpot_num()

    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100 # 포아송과 기하분포의 cdf로 계산된 확률
        
        # 20% 기준으로 구간 설정하여 분포확인하기
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)  # 구간별 카운트를 0으로 초기화
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:  # 숫자가 해당 구간에 속하는지 확인
                    interval_counts[i] += 1
                    break  # 해당 구간에 숫자가 속하면 다음 숫자로 넘어감
        # jackpot_num_list의 길이가 0보다 큰 경우에만 평균을 계산합니다.
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0  # 또는 적절한 기본값
        
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)        
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else: # 이렇게도 되는구나 신기하다.
            start = -7  # 시작 범위
            end = 7     # 끝 범위
            gap = 0.5   # 간격
            value = 2   # 초기 대응값
            # -7부터 7까지 0.5 간격으로 범위 설정
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break  # 조건에 맞는 값을 찾으면 반복문 탈출
                start += gap  # 다음 범위로 업데이트
                value += 1  # 다음 값으로 업데이트
        
        #사라지는구간 찾기
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None  # 초기값 설정(없으면 오류)
        if len(jackpot_num_list) >= 6:  # 6번째 요소가 존재하는지 확인
            sixth_element = jackpot_num_list[5]  # 6번째 요소를 가져옴
            for index, (start, end) in enumerate(ranges, start=1):  # enumerate를 사용하여 각 구간의 인덱스를 얻습니다.
                if start <= sixth_element <= end:
                    forget = index

        
        print("\n")
        #print(bul_r)
        print(f"사라지는구간{forget}")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        #print(bul)
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        #print(f"이전 잭팟까지의 횟수 = {jackpot_num_list[-5:]}")
        
        save_betting_to_file(interval_counts, probability, bul, forget, previous_betting) # txt(csv)파일로 저장하는 코드

        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()

# 2. csv로 저장하는 코드

In [ ]:
import random
import math
import csv

# CSV 파일에 데이터를 저장하는 함수
def save_betting_to_csv(interval_counts, probability, bul, forget, betting):
    with open("betting_history.csv", "a", newline='') as file:  # CSV 파일을 append 모드로 엽니다.
        writer = csv.writer(file)  # csv.writer 객체를 생성합니다.
        # writerow 메소드를 사용하여 데이터를 한 줄씩 파일에 씁니다.
        writer.writerow([interval_counts, "{:.2f}".format(probability), bul, forget, betting])

total = int(input("얼마를 충전하시겠습니까? "))

jackpot_num_list = []


previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:

        num = random.randint(1, 125)
        if num == 1:
                    return jackpot_num
        else:
            jackpot_num += 1


def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget # global로 해주어야 밖으로 반환됨

    jackpot_num = make_jackpot_num()

    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100 # 포아송과 기하분포의 cdf로 계산된 확률
        
        # 20% 기준으로 구간 설정하여 분포확인하기
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)  # 구간별 카운트를 0으로 초기화
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:  # 숫자가 해당 구간에 속하는지 확인
                    interval_counts[i] += 1
                    break  # 해당 구간에 숫자가 속하면 다음 숫자로 넘어감
        # jackpot_num_list의 길이가 0보다 큰 경우에만 평균을 계산합니다.
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0  # 또는 적절한 기본값
        
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)        
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else: # 이렇게도 되는구나 신기하다.
            start = -7  # 시작 범위
            end = 7     # 끝 범위
            gap = 0.5   # 간격
            value = 2   # 초기 대응값
            # -7부터 7까지 0.5 간격으로 범위 설정
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break  # 조건에 맞는 값을 찾으면 반복문 탈출
                start += gap  # 다음 범위로 업데이트
                value += 1  # 다음 값으로 업데이트
        
        #사라지는구간 찾기
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None  # 초기값 설정(없으면 오류)
        if len(jackpot_num_list) >= 6:  # 6번째 요소가 존재하는지 확인
            sixth_element = jackpot_num_list[5]  # 6번째 요소를 가져옴
            for index, (start, end) in enumerate(ranges, start=1):  # enumerate를 사용하여 각 구간의 인덱스를 얻습니다.
                if start <= sixth_element <= end:
                    forget = index

        
        print("\n")
        #print(bul_r)
        print(f"사라지는구간{forget}")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        #print(bul)
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        #print(f"이전 잭팟까지의 횟수 = {jackpot_num_list[-5:]}")
        
        save_betting_to_csv(interval_counts, probability, bul, forget, previous_betting)  # CSV 파일에 저장

        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()

# 3. csv파일로 저장하며 수정하는 게임코드

In [ ]:
import random
import math
import csv

# CSV 파일에 데이터를 저장하는 함수
def save_betting_to_csv(interval_counts, probability, bul, forget, betting):
    with open("betting_history.csv", "a", newline='') as file:  # CSV 파일을 append 모드로 엽니다.
        writer = csv.writer(file)  # csv.writer 객체를 생성합니다.
        # writerow 메소드를 사용하여 데이터를 한 줄씩 파일에 씁니다.
        writer.writerow([interval_counts, "{:.2f}".format(probability), bul, forget, betting])

total = int(input("얼마를 충전하시겠습니까? "))

jackpot_num_list = []


previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:

        num = random.randint(1, 125)
        if num == 1:
                    return jackpot_num
        else:
            jackpot_num += 1


def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget # global로 해주어야 밖으로 반환됨

    jackpot_num = make_jackpot_num()

    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100 # 포아송과 기하분포의 cdf로 계산된 확률
        
        # 20% 기준으로 구간 설정하여 분포확인하기
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)  # 구간별 카운트를 0으로 초기화
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:  # 숫자가 해당 구간에 속하는지 확인
                    interval_counts[i] += 1
                    break  # 해당 구간에 숫자가 속하면 다음 숫자로 넘어감
        # jackpot_num_list의 길이가 0보다 큰 경우에만 평균을 계산합니다.
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0  # 또는 적절한 기본값
        
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)        
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else: # 이렇게도 되는구나 신기하다.
            start = -7  # 시작 범위
            end = 7     # 끝 범위
            gap = 0.5   # 간격
            value = 2   # 초기 대응값
            # -7부터 7까지 0.5 간격으로 범위 설정
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break  # 조건에 맞는 값을 찾으면 반복문 탈출
                start += gap  # 다음 범위로 업데이트
                value += 1  # 다음 값으로 업데이트
        
        #사라지는구간 찾기
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None  # 초기값 설정(없으면 오류)
        if len(jackpot_num_list) >= 6:  # 6번째 요소가 존재하는지 확인
            sixth_element = jackpot_num_list[5]  # 6번째 요소를 가져옴
            for index, (start, end) in enumerate(ranges, start=1):  # enumerate를 사용하여 각 구간의 인덱스를 얻습니다.
                if start <= sixth_element <= end:
                    forget = index

        
        print("\n")
        #print(bul_r)
        print(f"사라지는구간{forget}")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        #print(bul)
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        #print(f"이전 잭팟까지의 횟수 = {jackpot_num_list[-5:]}")
        
        save_betting_to_csv(interval_counts, probability, bul, forget, previous_betting)  # CSV 파일에 저장
        
        # 파일 수정하는 부분
        def modify_betting_in_csv(file_name, target_values):
            data_modified = False
            data = []

            # 파일에서 데이터 읽기
            with open(file_name, 'r') as file:
                reader = csv.reader(file)
                for row in reader:
                    data.append(row)

            # 사용자 입력을 받아 해당하는 행 찾기
            for i, row in enumerate(data):
                interval_counts, probability, bul, forget = target_values  # 사용자가 입력한 값

                # CSV 파일에서 읽은 값과 비교 (문자열과 비교를 위해 CSV에서 읽은 값을 문자열로 변환)
                if (row[0] == interval_counts and "{:.2f}".format(float(row[1])) == probability
                        and row[2] == bul and row[3] == forget):
                    print(f"Found a matching row: {row}")
                    print(f"Current betting value: {row[4]}")
                    # 사용자에게 새로운 배팅 금액을 입력 받음
                    new_betting = input("Enter new betting value (leave blank to keep current value): ")
                    if new_betting:
                        data[i][4] = new_betting  # 입력 받은 값으로 변경
                        data_modified = True  # 데이터가 수정됨을 표시

            # 변경된 데이터를 파일에 다시 쓰기
            if data_modified:
                with open(betting_history.csv, 'w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerows(data)
                    print("File updated successfully.")
            else:
                print("No matching row found or no change made.")

        # 사용자 입력 예시 (이 부분을 사용자의 실제 입력으로 대체해야 함)
        interval_counts_input = "[0, 1, 0, 0, 0]"  # 예를 들어 사용자가 입력한 구간분포
        probability_input = "25.12"  # 예를 들어 사용자가 입력한 확률
        bul_input = "2"  # 예를 들어 사용자가 입력한 bul 값
        forget_input = "1"  # 예를 들어 사용자가 입력한 forget 값
        # 파일 이름과 타깃 값들을 지정하여 함수 호출
        modify_betting_in_csv("betting_history.csv", [interval_counts_input, probability_input, bul_input, forget_input])
        
        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()

# 4

In [ ]:
import random
import math
import csv

# CSV 파일에 데이터를 저장하는 함수
def save_betting_to_csv(interval_counts, probability, bul, forget, betting):
    with open("betting_history.csv", "a", newline='') as file:  # CSV 파일을 append 모드로 엽니다.
        writer = csv.writer(file)  # csv.writer 객체를 생성합니다.
        # writerow 메소드를 사용하여 데이터를 한 줄씩 파일에 씁니다.
        writer.writerow([interval_counts, "{:.2f}".format(probability), bul, forget, betting])

# 파일 수정하는 함수
def modify_betting_in_csv(new_value):
    with open("betting_history.csv", 'r', newline='') as file:
        data = list(csv.reader(file))
    with open("betting_history.csv", 'w', newline='') as file:
        writer = csv.writer(file)
        for row in data:
            row[4] = new_value  # 다섯 번째 요소 변경
        writer.writerows(data)

total = int(input("얼마를 충전하시겠습니까? "))

jackpot_num_list = []

previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:
        num = random.randint(1, 125)
        if num == 1:
            return jackpot_num
        else:
            jackpot_num += 1

def slot_machine():
    global total, previous_betting, jackpot_num_list
    jackpot_num = make_jackpot_num()

    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue
        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100 # 포아송과 기하분포의 cdf로 계산된 확률

        # 20% 기준으로 구간 설정하여 분포확인하기
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)  # 구간별 카운트를 0으로 초기화
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:  # 숫자가 해당 구간에 속하는지 확인
                    interval_counts[i] += 1
                    break  # 해당 구간에 숫자가 속하면 다음 숫자로 넘어감
        # jackpot_num_list의 길이가 0보다 큰 경우에만 평균을 계산합니다.
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0  # 또는 적절한 기본값
        
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)        
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else: # 이렇게도 되는구나 신기하다.
            start = -7  # 시작 범위
            end = 7     # 끝 범위
            gap = 0.5   # 간격
            value = 2   # 초기 대응값
            # -7부터 7까지 0.5 간격으로 범위 설정
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break  # 조건에 맞는 값을 찾으면 반복문 탈출
                start += gap  # 다음 범위로 업데이트
                value += 1  # 다음 값으로 업데이트
        
        #사라지는구간 찾기
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None  # 초기값 설정(없으면 오류)
        if len(jackpot_num_list) >= 6:  # 6번째 요소가 존재하는지 확인
            sixth_element = jackpot_num_list[5]  # 6번째 요소를 가져옴
            for index, (start, end) in enumerate(ranges, start=1):  # enumerate를 사용하여 각 구간의 인덱스를 얻습니다.
                if start <= sixth_element <= end:
                    forget = index

        
        print("\n")
        #print(bul_r)
        print(f"사라지는구간{forget}")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        #print(bul)
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        #print(f"이전 잭팟까지의 횟수 = {jackpot_num_list[-5:]}")
        
        # 사용자로부터 입력받는 부분
        input_1 = interval_counts
        input_2 = probability
        input_3 = bul
        input_4 = forget

        # CSV 파일 경로 설정
        filename = "betting_history.csv"

        # CSV 파일 열기 및 읽기
        with open(filename, 'r', newline='') as file:
            reader = csv.reader(file)
            for row in reader:
                # 행의 앞 네 개의 특징과 입력값 비교
                if (row[0] == input_1 and row[1] == input_2 and
                        row[2] == input_3 and row[3] == input_4):
                    # 일치하는 경우
                    modify_betting_in_csv(interval_counts, probability, bul, forget, previous_betting)
                    break
            else:
                save_betting_to_csv(interval_counts, probability, bul, forget, previous_betting)  # CSV 파일에 저장

        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            break

while True:
    slot_machine()


In [ ]:
import random
import math
import csv

# CSV 파일에 데이터를 저장하는 함수
def save_betting_to_csv(interval_counts, probability, bul, forget, betting):
    with open("betting_history.csv", "a", newline='') as file:  # 'a' 모드로 변경
        writer = csv.writer(file)
        # 문자열로 인식되는 probability 문제 해결을 위해 float 변환을 추가
        writer.writerow([interval_counts, "{:.2f}".format(float(probability)), bul, forget, betting])

# 파일 수정하는 함수
def modify_betting_in_csv(interval_counts, probability, bul, forget, betting):
    data = []
    with open("betting_history.csv", 'r', newline='') as file:
        data = list(csv.reader(file))
    with open("betting_history.csv", 'w', newline='') as file:
        writer = csv.writer(file)
        for row in data:
            if row[0] == interval_counts and row[1] == "{:.2f}".format(float(probability)) and row[2] == str(bul) and row[3] == str(forget):
                row[4] = str(betting)  # 다섯 번째 요소 변경
        writer.writerows(data)

total = int(input("얼마를 충전하시겠습니까? "))
jackpot_num_list = []
previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:
        num = random.randint(1, 125)
        if num == 1:
            return jackpot_num
        else:
            jackpot_num += 1

def slot_machine():
    global total, previous_betting, jackpot_num_list
    jackpot_num = make_jackpot_num()
    play_num = 0
    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue
        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100

        # 20% 기준으로 구간 설정하여 분포확인하기
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)  # 구간별 카운트를 0으로 초기화
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:  # 숫자가 해당 구간에 속하는지 확인
                    interval_counts[i] += 1
                    break  # 해당 구간에 숫자가 속하면 다음 숫자로 넘어감

        # jackpot_num_list의 길이가 0보다 큰 경우에만 평균을 계산합니다.
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0  # 또는 적절한 기본값

        # 불균도 계산
        bul_r = round((sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list),2)
        if bul_r < -7:
            bul = 1
        elif bul_r >= 7:
            bul = 30
        else:
            start = -7  # 시작 범위
            end = 7     # 끝 범위
            gap = 0.5   # 간격
            value = 2   # 초기 대응값
            # -7부터 7까지 0.5 간격으로 범위 설정
            while start < end:
                if start <= bul_r < start + gap:
                    bul = value
                    break  # 조건에 맞는 값을 찾으면 반복문 탈출
                start += gap  # 다음 범위로 업데이트
                value += 1  # 다음 값으로 업데이트

        #사라지는구간 찾기
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = None  # 초기값 설정(없으면 오류)
        if len(jackpot_num_list) >= 6:  # 6번째 요소가 존재하는지 확인
            sixth_element = jackpot_num_list[5]  # 6번째 요소를 가져옴
            for index, (start, end) in enumerate(ranges, start=1):  # enumerate를 사용하여 각 구간의 인덱스를 얻습니다.
                if start <= sixth_element <= end:
                    forget = index

        # CSV 파일 존재 여부 확인 및 데이터 읽고 쓰기
        filename = "betting_history.csv"
        try:
            with open(filename, 'r', newline='') as file:
                reader = csv.reader(file)
                found = False
                for row in reader:
                    if (row[0] == str(interval_counts) and row[1] == "{:.2f}".format(float(probability)) and
                            row[2] == str(bul) and row[3] == str(forget)):
                        found = True
                        modify_betting_in_csv(str(interval_counts), "{:.2f}".format(float(probability)), bul, forget, previous_betting)
                        break
                if not found:
                    save_betting_to_csv(str(interval_counts), "{:.2f}".format(float(probability)), bul, forget, previous_betting)
        except FileNotFoundError:
            # 파일이 없는 경우 새로 생성
            save_betting_to_csv(str(interval_counts), "{:.2f}".format(float(probability)), bul, forget, previous_betting)

        print("\n")
        print(f"사라지는 구간: {forget}")
        print(f"현재 {play_num}회 배팅 중입니다. 잔액: {total}")
        print(f"구간 분포: {interval_counts}, 당첨 확률: {probability:.2f}%, 불균도: {bul_r}, 평균: {average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전: {jackpot_num_list[-5:]}")

        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! 잔액: {total}, 평균 잭팟 번호: {average}")
            break

while True:
    slot_machine()

In [ ]:
import csv

def make_num():  # 함수 선언 수정
    # 네 개의 숫자 입력 받음
    numbers = input("네 개의 숫자를 공백으로 구분하여 입력하세요: ").split()

    # 정확히 네 개의 숫자가 입력되었는지 확인
    if len(numbers) == 4 and all(n.isdigit() for n in numbers):
        # 문자열 입력을 정수로 변환
        numbers = [int(n) for n in numbers]

        # 쓰기 모드로 파일 열기
        with open('numbers.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(numbers)  # CSV의 한 행으로 숫자 쓰기
        print("numbers.csv 파일에 숫자가 저장되었습니다.")
    else:
        print("정확히 네 개의 숫자를 입력해주세요.")

while True:
    make_num()  # 함수 호출
    # 사용자가 원할 때 루프를 빠져나갈 수 있는 방법 추가(예시)
    if input("종료하려면 'q'를 입력하세요: ") == 'q':
        break

## csv파일과 상호작용하며 중복된것 발견하는것까지했다.

In [ ]:
import csv

def make_num():
    # 네 개의 숫자 입력 받음
    numbers = input("네 개의 숫자를 공백으로 구분하여 입력하세요: ").split()

    # 정확히 네 개의 숫자가 입력되었는지 확인
    if len(numbers) == 4 and all(n.isdigit() for n in numbers):
        # 문자열 입력을 정수로 변환
        numbers = [int(n) for n in numbers]

        # 기존에 저장된 숫자들을 확인
        existing_numbers = []
        try:
            with open('numbers.csv', 'r', newline='') as file:
                reader = csv.reader(file)
                for row in reader:
                    existing_numbers.append([int(num) for num in row])
        except FileNotFoundError:
            pass  # 파일이 없으면 빈 리스트를 유지

        # 입력된 숫자 배열이 기존에 존재하는지 확인
        if numbers in existing_numbers:
            print("이미 입력된 숫자 배열입니다.")
        else:
            # 쓰기 모드로 파일 열기
            with open('numbers.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(numbers)  # CSV의 한 행으로 숫자 쓰기
            print("numbers.csv 파일에 숫자가 저장되었습니다.")
    else:
        print("정확히 네 개의 숫자를 입력해주세요.")

while True:
    make_num()  # 함수 호출
    # 사용자가 원할 때 루프를 빠져나갈 수 있는 방법 추가
    if input("종료하려면 'q'를 입력하세요: ") == 'q':
        break

## 세번째 숫자까지의 배열을 보고 같은것이 있으면 네번째 숫자를 알려주는것까지 가능

In [ ]:
import csv

def make_num():
    # 네 개의 숫자 입력 받음
    numbers = input("네 개의 숫자를 공백으로 구분하여 입력하세요: ").split()

    # 정확히 네 개의 숫자가 입력되었는지 확인
    if len(numbers) == 4 and all(n.isdigit() for n in numbers):
        # 문자열 입력을 정수로 변환
        numbers = [int(n) for n in numbers]

        # 기존에 저장된 숫자들을 확인 #여기부터
        existing_numbers = []
        try:
            with open('numbers.csv', 'r', newline='') as file:
                reader = csv.reader(file)
                for row in reader:
                    existing_numbers.append([int(num) for num in row])
        except FileNotFoundError:
            pass  # 파일이 없으면 빈 리스트를 유지 #여기까지 한 역할덩어리

        # 입력된 숫자 배열의 첫 세 요소와 기존에 존재하는 배열의 첫 세 요소가 일치하는지 확인
        for existing in existing_numbers: # existing으로 숫자를 지칭 (모든 existing에 대하여 시행)
            if existing[:3] == numbers[:3]:
                print(f"입력하신 배열의 첫 세 숫자와 일치하는 배열이 존재합니다: {existing}, 네 번째 숫자는 {existing[3]}입니다.")
                return  # 일치하는 배열을 찾으면 함수 종료

        # 일치하는 배열이 없으면 새로운 숫자 배열을 파일에 추가
        with open('numbers.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(numbers)  # CSV의 한 행으로 숫자 쓰기
        print("numbers.csv 파일에 숫자가 저장되었습니다.")
    else:
        print("정확히 네 개의 숫자를 입력해주세요.")

while True:
    make_num()  # 함수 호출
    # 사용자가 원할 때 루프를 빠져나갈 수 있는 방법 추가
    if input("종료하려면 'q'를 입력하세요: ") == 'q':
        break

## 원하는 바를 달성했다 세번째 요소까지 비교하여 같은것 찾고 이를 바꿀지 말지 결정하여 바꾸는기능까지 추가됨

In [ ]:
import csv

def make_num():
    # 네 개의 숫자 입력 받음
    numbers = input("네 개의 숫자를 공백으로 구분하여 입력하세요: ").split()

    # 정확히 네 개의 숫자가 입력되었는지 확인
    if len(numbers) == 4 and all(n.isdigit() for n in numbers):
        # 문자열 입력을 정수로 변환
        numbers = [int(n) for n in numbers]

        # 기존에 저장된 숫자들을 확인
        existing_numbers = []
        try:
            with open('numbers.csv', 'r', newline='') as file:
                reader = csv.reader(file)
                existing_numbers = [ [int(num) for num in row] for row in reader]
        except FileNotFoundError:
            pass  # 파일이 없으면 빈 리스트를 유지

        # 입력된 숫자 배열의 첫 세 요소와 기존에 존재하는 배열의 첫 세 요소가 일치하는지 확인
        for idx, existing in enumerate(existing_numbers):
            if existing[:3] == numbers[:3]:
                print(f"입력하신 배열의 첫 세 숫자와 일치하는 배열이 존재합니다: {existing}, 네 번째 숫자는 {existing[3]}입니다.")
                change = input("네 번째 숫자를 변경하시겠습니까? (yes/no): ").strip().lower()
                if change == 'yes':
                    new_number = input("새로운 네 번째 숫자를 입력하세요: ").strip()
                    if new_number.isdigit():
                        existing_numbers[idx][3] = int(new_number)
                        # 변경된 배열을 파일에 다시 쓰기
                        with open('numbers.csv', 'w', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerows(existing_numbers)
                        print(f"네 번째 숫자가 {new_number}로 변경되었습니다.")
                    else:
                        print("유효한 숫자를 입력하지 않았습니다.")
                return  # 변경 여부에 관계없이 함수 종료

        # 일치하는 배열이 없으면 새로운 숫자 배열을 파일에 추가
        with open('numbers.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(numbers)  # CSV의 한 행으로 숫자 쓰기
        print("numbers.csv 파일에 숫자가 저장되었습니다.")
    else:
        print("정확히 네 개의 숫자를 입력해주세요.")

while True:
    make_num()  # 함수 호출
    # 사용자가 원할 때 루프를 빠져나갈 수 있는 방법 추가
    if input("종료하려면 'q'를 입력하세요: ") == 'q':
        break

# 핸드폰으로 옮기는 코드변형

In [ ]:
import math

jackpot_num_list = []
play_num = 0

def slot_machine():
    global jackpot_num_list, play_num
    # 실제 환경에서 이 부분은 사용자 입력을 받아야 합니다.
    button = input("버튼을 누르세요 (1(꽝)/2(잭팟)): ")

    # 당첨 확률 계산
    probability = (1 - math.exp(-(play_num / 125))) * 100

    # 구간 분포 처리
    intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
    last_five = jackpot_num_list[-5:]
    interval_counts = [0] * len(intervals)
    for num in last_five:
        for i, interval in enumerate(intervals):
            if interval[0] <= num <= interval[1]:
                interval_counts[i] += 1
                break

    # 마지막 5개 요소 평균 처리
    average_5 = round(sum(last_five) / len(last_five)) if last_five else 0

    # 불균도 처리
    bul_r = (sum(jackpot_num_list) + play_num) / 125 - len(jackpot_num_list)

    # 사라지는 구간 처리
    ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
    forget = 0  # 기본값을 0으로 설정
    if len(jackpot_num_list) >= 6:
        sixth_element = jackpot_num_list[-6]
        for index, (start, end) in enumerate(ranges, start=1):
            if start <= sixth_element <= end:
                forget = index

    # 버튼 입력에 따라 처리
    if button == '1':
        play_num += 1
        print(f"현재 {play_num}회 배팅 중입니다.")
        print(f"구간 분포 {interval_counts}, 당첨 확률 {probability:.2f}%, 불균도 {bul_r:.2f}, 평균 {average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전 {jackpot_num_list[-5:]}, 사라지는 구간 {forget}")
    elif button == '2':
        jackpot_num_list.append(play_num)

while True:
    slot_machine()

# 이 코드를 앱으로 변형하면 끝난다.

In [ ]:
import math

jackpot_num_list = []
play_num = 0

def slot_machine():
    global jackpot_num_list, play_num

    # 사용자 입력
    button = input("버튼을 누르세요 (1(꽝)/2(잭팟)): ")

    # 배팅 횟수 증가
    play_num += 1

    # 당첨 확률 계산
    probability = (1 - math.exp(-(play_num / 125))) * 100

    # 구간 분포 처리
    intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
    last_five = jackpot_num_list[-5:]
    interval_counts = [0] * len(intervals)
    for num in last_five:
        for i, interval in enumerate(intervals):
            if interval[0] <= num <= interval[1]:
                interval_counts[i] += 1
                break

    # 마지막 5개 요소 평균 처리
    average_5 = round(sum(last_five) / len(last_five)) if last_five else 0

    # 불균도 처리
    bul = (sum(jackpot_num_list) + play_num) / 125 - len(jackpot_num_list)

    # 사라지는 구간 처리
    ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
    forget = 0  # 기본값을 0으로 설정
    if len(jackpot_num_list) >= 6:
        sixth_element = jackpot_num_list[-6]
        for index, (start, end) in enumerate(ranges, start=1):
            if start <= sixth_element <= end:
                forget = index

    # 버튼 입력에 따라 처리
    if button == '1':
        print(f"현재 {play_num}회 배팅 중입니다.")
        print(f"구간 분포 {interval_counts}, 당첨 확률 {probability:.2f}%, 불균도 {bul_r:.2f}, 평균 {average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전 {jackpot_num_list[-5:]}, 사라지는 구간 {forget}")
    elif button == '2':
        jackpot_num_list.append(play_num)
        print(f"축하합니다! 잭팟입니다! {play_num}회 만에 당첨되셨습니다!")
        play_num = 0

while True:
    slot_machine()

# Kotliin언어로 최적화하여 핸드폰 어플로 만든다.

In [ ]:
import kotlin.math.exp
import kotlin.math.roundToInt

var jackpotNumList = mutableListOf<Int>()
var playNum = 0
var probability = 0.0
var bulR = 0.0
var average5 = 0
var forget = 0
val intervals = listOf(Pair(1, 28), Pair(29, 64), Pair(65, 115), Pair(116, 200), Pair(200, 1200))

fun slotMachine() {
    print("버튼을 누르세요 (1(꽝)/2(잭팟)): ")
    val buttonInput = readLine()

    if (buttonInput != null) {
        when (buttonInput) {
            "1" -> {
                playNum += 1
                probability = (1 - exp(-(playNum.toDouble() / 125))) * 100
                val lastFive = jackpotNumList.takeLast(5)
                var intervalCounts = IntArray(intervals.size) { 0 }
                for (num in lastFive) {
                    for ((i, interval) in intervals.withIndex()) {
                        if (num in interval.first..interval.second) {
                            intervalCounts[i] += 1
                            break
                        }
                    }
                }
                average5 = if (lastFive.isNotEmpty()) {
                    lastFive.sum().toDouble() / lastFive.size
                } else {
                    0.0
                }.roundToInt()
                bulR = (jackpotNumList.sum() + playNum.toDouble()) / 125.0 - jackpotNumList.size

                if (jackpotNumList.size >= 6) {
                    val sixthElement = jackpotNumList[jackpotNumList.size - 6]
                    for ((index, range) in intervals.withIndex()) {
                        if (sixthElement in range.first..range.second) {
                            forget = index + 1
                            break
                        }
                    }
                }
                println("현재 ${playNum}회 배팅 중입니다.")
                println("구간 분포 ${intervalCounts.joinToString()}, 당첨 확률 ${String.format("%.2f", probability)}%, 불균도 ${String.format("%.2f", bulR)}, 평균 $average5")
                println("(1,28)(29,64)(65,115)(116,200)(201), 이전 ${jackpotNumList.takeLast(5)}, 사라지는 구간 $forget")
            }
            "2" -> {
                playNum += 1
                jackpotNumList.add(playNum)
                println("축하합니다! 잭팟입니다! ${playNum}회 만에 당첨되셨습니다!")
                playNum = 0
            }
        }
    }
}

fun main() {
    while (true) {
        slotMachine()
    }
}

# 실전 이후 수정

In [ ]:
import random
import math

total = int(input("얼마를 충전하시겠습니까? "))
jackpot_num_list = []
previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:
        num = random.randint(1, 250)
        if num == 1:
            return jackpot_num
        else:
            jackpot_num += 1

def slot_machine():
    global total, previous_betting, jackpot_num_list, interval_counts, probability, bul, forget
    jackpot_num = make_jackpot_num()
    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/250)))*100
        # 구간분포 처리하는 부분
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:
                    interval_counts[i] += 1
                    break
                    
        # 마지막 5개 요소평균 처리하는 부분
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0
        
        # 불균도 처리하는 부분
        bul_r = (sum(jackpot_num_list) + play_num) / 125 - len(jackpot_num_list)
        bul = int(bul_r)  # 소수점 아래를 제거하고 정수 부분만 남깁니다.
        
        # 사라지는 구간 처리하는 부분
        ranges = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        forget = 0  # 기본값을 0으로 설정합니다.
        if len(jackpot_num_list) >= 5:
            fifth_from_last_element = jackpot_num_list[-5]  # 리스트의 뒤에서 다섯 번째 요소
            for index, (start, end) in enumerate(ranges, start=1):
                if start <= fifth_from_last_element <= end:
                    forget = index
                    break  # 해당 구간을 찾으면 반복문 종료


        print("\n")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {bul_r:.2f} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]} 사라지는구간{forget}")
        
        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()
